># **Data Cleaning**

In [26]:
import numpy as np
import pandas as pd

#### Importing the dataset

In [27]:
df = pd.read_csv(filepath_or_buffer="../data/raw/medical_costs.csv")
df.head()

,Age,Sex,BMI,Children,Smoker,Region,Medical Cost
0,58,male,15.6,2,yes,northwest,17907.54
1,24,male,29.8,0,yes,northeast,16312.64
2,50,male,29.0,5,no,northwest,6819.21
3,35,male,34.0,1,no,southeast,5247.87
4,31,female,17.6,3,yes,southeast,17525.49


#### Changing columns to lower case

In [28]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

#### Removing duplicate entries if present

In [29]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           10000 non-null  int64  
 1   sex           10000 non-null  object 
 2   bmi           10000 non-null  float64
 3   children      10000 non-null  int64  
 4   smoker        10000 non-null  object 
 5   region        10000 non-null  object 
 6   medical_cost  10000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 547.0+ KB


In [31]:
df.describe()

,age,bmi,children,medical_cost
count,10000.000000,10000.00000,10000.000000,10000.000000
mean,41.678400,27.40301,2.501700,11898.932216
std,13.807724,7.22896,1.701672,6073.875834
min,18.000000,15.00000,0.000000,3617.090000
25%,30.000000,21.10000,1.000000,5909.925000
50%,42.000000,27.40000,2.000000,7957.430000
75%,54.000000,33.70000,4.000000,17931.962500
max,65.000000,40.00000,5.000000,20268.210000


#### Checking for no. of unique elements in each column 

In [32]:
df.nunique()

age               48
sex                2
bmi              251
children           6
smoker             2
region             4
medical_cost    9912
dtype: int64

#### Checking for null values in data

In [34]:
df.isnull().sum()

age             0
sex             0
bmi             0
children        0
smoker          0
region          0
medical_cost    0
dtype: int64

In [33]:
df.replace(
    to_replace=["", " ", "NA", "N/A", "None", "-", "nan", "null", "NULL"],
    value=np.nan,
    inplace=True,
)

#### Handling missing values if outliers are present

In [35]:
def outliers(data):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lb = q1 - (iqr * 1.5)
    ub = q3 + (iqr * 1.5)
    outliers = data[(data < lb) | (data > ub)]
    return outliers


bmi = outliers(df["bmi"])
mc = outliers(df["medical_cost"])


def fill_missing_value(df, column, outliers):
    if not outliers.empty:
        df[column] = df[column].fillna(df[column].median())
    else:
        df[column] = df[column].fillna(df[column].mean())


fill_missing_value(df, "bmi", bmi)
fill_missing_value(df, "medical_cost", mc)

#### Exporting cleaned data set

In [36]:
df.to_csv("../data/interim/medical_cost_interim.csv", index=False)